# An Introduction to Models in Pyro

THe basic unit of probabilistic programs is the *stochastic function*. This is an arbitrary Python callable that combines two ingredients: 

* deterministic Python code; and 
* primitive stochastic functions that call a random number generator

Concretely, a stochastic function can be any Python object with a `__call__()` method, like a function, a method, or PyTorch `nn.Module`.

Throughout the tutorials and documentation, we will often call stochasti functions *models*, since stochastic functions can be used to represent simplified or abstract descriptions of a process by which data are generated. Expressing models as stochastic functions means that models can be composed, reused, imported, and serialized just like regulat Python callables. 


In [ ]:
import torch
import pyro

pyro.set_rng_seed(101)

# Primitive Stochastic Functions

Primitive stochastic fnuctions, or distributions, are an important class of stochastic functions for which we can explicility compute the probability of the outputs given the inputs. As of PyTorch 0.4 and Pyro 0.2, Pyro uses PyTorch's 'distribution libaray'. You can also create custom distributions using 'transforms'. 

Using primitive stochastic functions is easy. For example, to draw a sample `x` from the unit normal distribution $ \mathcal{N}(0, 1)$ we do the following: 




In [1]:
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

NameError: name &#39;torch&#39; is not defined

Here, `torch.distribution.Normal` is an instance of the `Distribution` class that takes parameters and provides sample and score methods. Pyro's distribution library `pyro.distributions` is a thin wrapper around `torch.distribution` because we want to make use of PyTorch's fast tensor math and autograd capabilities during inference.

# A Simple Model 

All probabilistic programs are built up by composing primitive stochastic functions and determinstic computation. Since we're ultimately insterested in probablistic programming because we want to model things in the real world let's start with a model of something concrete. 

Let's suppose we have bunch of data with daily mean temperatures and cloud cover. We want to reason about how termperature interacts with whether it was sunny or cloudy. A simple sotchastic funciton that describes how that data might have been generated is given by: 

In [ ]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

Let's go through this line-by-line First, in lines 2 we define a binary random variables 'cloudy' which is given by a draw from the Bernoulli distribution with parameter `0.3`. Since the Bernoulli distributions reuturn `0` or `1`s, in line 3 we convert the value to `cloudy` to a string so that return values of `weather` are easier to parse. So according to this model 30% of the time it's cloudy and 70% of the time it's sunny 

IN lined 4-5, we define the parameters we're going to use to sample the temperature in lines6. These parameters depend on the particular value of `cloudy` we sampled in line 2. For example, the mean temperature is 55 degrees (F) on cloudy days and 75 degrees (F) on sunny days. Finally we return the two values `cloudy` and `temp` in line 7. 

However, `weather` is entirely independent of Pyro - it only call Pytorch. We need to turn it into a Pyro pogram if we want to use thi model for anything other than sampling fake data. 

# The `pyro.sample` Primitive


To turn `weather` into a Pyro program, we’ll replace the `torch.distributions` with `pyro.distributions` and the `.sample()` and `.rsample()` calls with calls to `pyro.sample`, one of the core language primitives in Pyro. Using `pyro.sample` is as simple as calling a primitive stochastic function with one important difference:


In [ ]:
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x)

Just like a direct call to `torch.distributions.Normal().rsample()`, this returns a sample from the unit normal distribution. The crucial difference is that this sample is named. Pyro’s backend uses these names to uniquely identify sample statements and change their behavior at runtime depending on how the enclosing stochastic function is being used. As we will see, this is how Pyro can implement the various manipulations that underlie inference algorithms.

Now that we’ve introduced `pyro.sample` and `pyro.distributions` we can rewrite our simple model as a Pyro program:

In [ ]:
def weather():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

Procedurally, `weather()` is still a non-deterministic Python callable that returns two random samples. Because the randomness is now invoked with `pyro.sample`, however, it is much more than that. In particular weather() specifies a joint probability distribution over two named random variables: `cloudy` and `temp`. As such, it defines a probabilistic model that we can reason about using the techniques of probability theory. For example we might ask: if I observe a temperature of 70 degrees, how likely is it to be cloudy? How to formulate and answer these kinds of questions will be the subject of the next tutorial.

# Universality: Stochastic Recursion, Higher-order Stochastic functions, and Random Control Flow

We’ve now seen how to define a simple model. Building off of it is easy. For example:

In [ ]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

This kind of modularity, familiar to any programmer, is obviously very powerful. But is it powerful enough to encompass all the different kinds of models we’d like to express?

It turns out that because Pyro is embedded in Python, stochastic functions can contain arbitrarily complex deterministic Python and randomness can freely affect control flow. For example, we can construct recursive functions that terminate their recursion nondeterministically, provided we take care to pass `pyro.sample` unique sample names whenever it’s called. For example we can define a geometric distribution that counts the number of failures until the first success like so:

In [ ]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

Note that the names `x_0`, `x_1`, etc., in `geometric()` are generated dynamically and that different executions can have different numbers of named random variables.

We are also free to define stochastic functions that accept as input or produce as output other stochastic functions:

In [ ]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.))

Here `make_normal_normal()` is a stochastic function that takes one argument and which, upon execution, generates three named random variables.

The fact that Pyro supports arbitrary Python code like this—iteration, recursion, higher-order functions, etc.—in conjuction with random control flow means that Pyro stochastic functions are universal, i.e. they can be used to represent any computable probability distribution. As we will see in subsequent tutorials, this is incredibly powerful.

It is worth emphasizing that this is one reason why Pyro is built on top of PyTorch: dynamic computational graphs are an important ingredient in allowing for universal models that can benefit from GPU-accelerated tensor math.

# Next Steps

We’ve shown how we can use stochastic functions and primitive distributions to represent models in Pyro. In order to learn models from data and reason about them we need to be able to do inference. This is the subject of the next tutorial.
